<a href="https://colab.research.google.com/github/djalil21/sentiment-analysis-algerian-dialect/blob/master/BERTs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install and import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers datasets emoji

In [ ]:
!pip install --upgrade accelerate

In [ ]:
!apt-get install git-lfs

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import tensorflow as tf
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    DataCollatorWithPadding, 
    TrainingArguments, 
    Trainer, 
    pipeline
)
import re
import numpy as np
import pandas as pd
from emoji import EMOJI_DATA
import unicodedata
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric
import pyarrow as pa

# Data

In [ ]:
file_name="DZ.csv"
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PFE/' +file_name,names=["text","label"])

#df =df.sample(n=5000)

In [ ]:
df.head(10)

,text,label
0,اسوء هاتف !,negative
1,عيوب لي فيه يحمى بزاف وباتري عيانة فيه في كامي...,negative
2,صراحة ميصلحش شريتو وندمة,negative
3,خاوتي سمحولي بصح هاتف 0 وثقيل فالريزو,negative
4,أخي الهاتف راه عندي و يسخن بزاف واش نديرله,negative
5,يودي عندي هاذ برطابل هو مليح بصح كان دير بيه م...,negative
6,يسخون,negative
7,بعد عامين من الاستخدام الريزو ناقص بزاف في الا...,negative
8,عندي هذا الهاتف صلى شعل ويطفى وحدو ما السبب؟,negative
9,شريت هذا كابة بصح كي نعيط بيه منقدرش نكوبي حتى...,negative


In [ ]:
df = df.dropna(axis=0)

In [ ]:
df = df[df.label != "neutral"]

In [ ]:
df.label.value_counts()

positive    9748
negative    5574
Name: label, dtype: int64

In [ ]:
# search your emoji
def is_emoji(s):
    return s in EMOJI_DATA

def add_space(text):
  result = ''
  for char in text:
    if is_emoji(char):
      result += ' '
    result += char
  return result.strip()

In [ ]:
# Define preprocessing util function
def text_preprocessing(text):


    text=str(text)
    # Normalize unicode encoding
    text = unicodedata.normalize('NFC', text)
    
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    #Remove URLs
    text = re.sub('http://\S+|https://\S+', '',text)

    ## Convert text to lowercases
    text = text.lower()

    #remove tashkeel
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)

    #remove repetetions
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')

    #remove special arab letters
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)

    #keep 2 repeat
    text = re.sub(r'(.)\1+', r'\1\1', text)

    #maximum length 512
    #if len(text) > 512:
     #  text = text[:512]

    #emoji treatment
    text = add_space(text)

    return text

In [ ]:
df['text']=df.text.apply(text_preprocessing)

In [ ]:
df['label'] = df.label.map({"negative":0,"positive":1})

In [ ]:
df.head()

,text,label
0,اسوء هاتف !,0
1,عيوب لي فيه يحمي بزاف وباتري عيانه فيه في كامي...,0
2,صراحه ميصلحش شريتو وندمه,0
3,خاوتي سمحولي بصح هاتف 0 وثقيل فالريزو,0
4,اخي الهاتف راه عندي و يسخن بزاف واش نديرله,0


In [ ]:
train, test = train_test_split(df, test_size = 0.2)

In [ ]:
train_ds = Dataset.from_pandas(train, split="train")
test_ds = Dataset.from_pandas(test, split="test")

# BERT base multilingual

In [ ]:
torch.cuda.empty_cache()

In [ ]:
NAME="bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(NAME)
model = AutoModelForSequenceClassification.from_pretrained(NAME, num_labels=2, id2label={0: 'negative', 1: 'positive'})

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"],truncation=True, padding='max_length',max_length=128)

In [ ]:
tokenized_train = train_ds.map(preprocess_function, batched=True)
tokenized_test = test_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/12257 [00:00<?, ? examples/s]

Map:   0%|          | 0/3065 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 12257
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}


In [ ]:
repo_name = NAME+"_2k"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=10,
   weight_decay=0.01,
   save_strategy="epoch",
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.475700
1000,0.353000
1500,0.308800
2000,0.233600
2500,0.217200
3000,0.168600
3500,0.147400
4000,0.129800
4500,0.109100
5000,0.076200


TrainOutput(global_step=7670, training_loss=0.16306101868640646, metrics={'train_runtime': 2955.3635, 'train_samples_per_second': 41.474, 'train_steps_per_second': 2.595, 'total_flos': 8062380513868800.0, 'train_loss': 0.16306101868640646, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

<ipython-input-51-da2ad6e6192f>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 0.9896340370178223,
 'eval_accuracy': 0.8518760195758565,
 'eval_f1': 0.8859869412355601,
 'eval_runtime': 23.0967,
 'eval_samples_per_second': 132.703,
 'eval_steps_per_second': 8.313,
 'epoch': 10.0}

In [ ]:
pipe = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
pipe.save_pretrained(repo_name)

In [ ]:
sentiment_model = pipeline('sentiment-analysis',model=repo_name)
sentiment_model(["عجبتني", "ماعجبتنيش"])

[{'label': 'POSITIVE', 'score': 0.6932519674301147},
 {'label': 'NEGATIVE', 'score': 0.6867501735687256}]

# Dziri BERT



In [ ]:
torch.cuda.empty_cache()

In [ ]:
NAME="alger-ia/dziribert"
tokenizer = AutoTokenizer.from_pretrained(NAME)
model = AutoModelForSequenceClassification.from_pretrained(NAME, num_labels=2, id2label={0: 'negative', 1: 'positive'})

Some weights of the model checkpoint at alger-ia/dziribert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at alger-ia/dziribert and are

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"],truncation=True, padding='max_length',max_length=128)

In [ ]:
tokenized_train = train_ds.map(preprocess_function, batched=True)
tokenized_test = test_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/12257 [00:00<?, ? examples/s]

Map:   0%|          | 0/3065 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 12257
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}


In [ ]:
repo_name = NAME+"_2k"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=10,
   weight_decay=0.01,
   save_strategy="epoch",
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.322200
1000,0.230400
1500,0.178000
2000,0.085200
2500,0.076300
3000,0.040100
3500,0.027500
4000,0.028200
4500,0.019800
5000,0.013800


TrainOutput(global_step=7670, training_loss=0.06938412503437703, metrics={'train_runtime': 2701.4809, 'train_samples_per_second': 45.371, 'train_steps_per_second': 2.839, 'total_flos': 8062380513868800.0, 'train_loss': 0.06938412503437703, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.028145670890808,
 'eval_accuracy': 0.8792822185970636,
 'eval_f1': 0.9053708439897699,
 'eval_runtime': 23.3311,
 'eval_samples_per_second': 131.37,
 'eval_steps_per_second': 8.229,
 'epoch': 10.0}

In [ ]:
pipe = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
pipe.save_pretrained(repo_name)

In [ ]:
sentiment_model = pipeline('sentiment-analysis',model=repo_name)
sentiment_model(["عجبتني", "ماعجبتنيش"])

[{'label': 'POSITIVE', 'score': 0.6932519674301147},
 {'label': 'NEGATIVE', 'score': 0.6867501735687256}]

# AraBERT



In [ ]:
torch.cuda.empty_cache()

In [ ]:
NAME="aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(NAME)
model = AutoModelForSequenceClassification.from_pretrained(NAME, num_labels=2, id2label={0: 'negative', 1: 'positive'})

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"],truncation=True, padding='max_length',max_length=128)

In [ ]:
tokenized_train = train_ds.map(preprocess_function, batched=True)
tokenized_test = test_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/12257 [00:00<?, ? examples/s]

Map:   0%|          | 0/3065 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 12257
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}


In [ ]:
repo_name = NAME+"_2k"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=10,
   weight_decay=0.01,
   save_strategy="epoch",
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.446400
1000,0.355300
1500,0.305800
2000,0.246800
2500,0.230300
3000,0.195200
3500,0.165000
4000,0.156200
4500,0.136700
5000,0.121200


TrainOutput(global_step=7670, training_loss=0.18257191209158966, metrics={'train_runtime': 2754.748, 'train_samples_per_second': 44.494, 'train_steps_per_second': 2.784, 'total_flos': 8062380513868800.0, 'train_loss': 0.18257191209158966, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.8820879459381104,
 'eval_accuracy': 0.8574225122349103,
 'eval_f1': 0.8890017780035561,
 'eval_runtime': 23.09,
 'eval_samples_per_second': 132.742,
 'eval_steps_per_second': 8.315,
 'epoch': 10.0}

In [ ]:
pipe = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
pipe.save_pretrained(repo_name)

In [ ]:
sentiment_model = pipeline('sentiment-analysis',model=repo_name)
sentiment_model(["عجبتني", "ماعجبتنيش"])

[{'label': 'POSITIVE', 'score': 0.6932519674301147},
 {'label': 'NEGATIVE', 'score': 0.6867501735687256}]

# MarBERT


In [ ]:
torch.cuda.empty_cache()

In [ ]:
NAME="UBC-NLP/MARBERTv2"
tokenizer = AutoTokenizer.from_pretrained(NAME)
model = AutoModelForSequenceClassification.from_pretrained(NAME, num_labels=2, id2label={0: 'negative', 1: 'positive'})

Some weights of the model checkpoint at UBC-NLP/MARBERTv2 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"],truncation=True, padding='max_length',max_length=128)

In [ ]:
tokenized_train = train_ds.map(preprocess_function, batched=True)
tokenized_test = test_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/12257 [00:00<?, ? examples/s]

Map:   0%|          | 0/3065 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 12257
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}


In [ ]:
repo_name = NAME+"_2k"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=10,
   weight_decay=0.01,
   save_strategy="epoch",
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.409100
1000,0.283400
1500,0.242200
2000,0.179400
2500,0.174000
3000,0.133400
3500,0.106300


Step,Training Loss
500,0.409100
1000,0.283400
1500,0.242200
2000,0.179400
2500,0.174000
3000,0.133400
3500,0.106300
4000,0.098600
4500,0.076000
5000,0.056800


TrainOutput(global_step=7670, training_loss=0.12702134143575697, metrics={'train_runtime': 2837.5495, 'train_samples_per_second': 43.196, 'train_steps_per_second': 2.703, 'total_flos': 8062380513868800.0, 'train_loss': 0.12702134143575697, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

<ipython-input-25-da2ad6e6192f>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 0.8705043792724609,
 'eval_accuracy': 0.8835236541598694,
 'eval_f1': 0.9108168873344991,
 'eval_runtime': 22.2771,
 'eval_samples_per_second': 137.585,
 'eval_steps_per_second': 8.619,
 'epoch': 10.0}

In [ ]:
pipe = pipeline('sentiment-analysis')

In [ ]:
pipe.save_pretrained(repo_name)

In [ ]:
sentiment_model = pipeline('sentiment-analysis',model=repo_name)
sentiment_model(["عجبتني", "ماعجبتنيش"])

[{'label': 'POSITIVE', 'score': 0.6932519674301147},
 {'label': 'NEGATIVE', 'score': 0.6867501735687256}]

# Roberta

In [ ]:
torch.cuda.empty_cache()

In [ ]:
NAME="xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(NAME)
model = AutoModelForSequenceClassification.from_pretrained(NAME, num_labels=2, id2label={0: 'negative', 1: 'positive'})

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"],truncation=True, padding='max_length',max_length=128)

In [ ]:
tokenized_train = train_ds.map(preprocess_function, batched=True)
tokenized_test = test_ds.map(preprocess_function, batched=True)

In [ ]:
tokenized_train

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}


In [ ]:
repo_name = NAME+"_2k"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=8,
   per_device_eval_batch_size=8,
   num_train_epochs=10,
   weight_decay=0.01,
   save_strategy="epoch",
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
pipe = pipeline('sentiment-analysis')

In [ ]:
pipe.save_pretrained(repo_name)

In [ ]:
sentiment_model = pipeline('sentiment-analysis',model=repo_name)
sentiment_model(["عجبتني", "ماعجبتنيش"])

In [ ]:
!rmdir UBC-NLP/

In [ ]:
!rm -r bert-base-multilingual-cased_2k/